In [1]:
#connecting to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-dfe06d24-418c-048f-6f80-675a6f3da83e)


In [3]:
!pip uninstall torch torchvision

Found existing installation: torch 1.13.0
Uninstalling torch-1.13.0:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-1.13.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? Y
  Successfully uninstalled torch-1.13.0
Found existing installation: torchvision 0.1.6
Uninstalling torchvision-0.1.6:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/torchvision-0.1.6.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torchvision/*
Proceed (Y/n)? Y
  Successfully uninstalled torchvision-0.1.6


In [4]:
!pip install torch===1.13.0 torchvision===0.1.6 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached torch-1.13.0-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
  Using cached https://download.pytorch.org/whl/torchvision-0.1.6-py3-none-any.whl (16 kB)
Using cached torch-1.13.0-cp311-cp311-manylinux1_x86_64.whl (890.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.5.2 requires torch>=2.0.0, but you have torch 1.13.0 which is incompatible.
fastai 2.7.19 requires torchvision>=0.11, but you have torchvision 0.1.6 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 1.13.0 which is incompatible.


In [5]:
import torch
import torchvision

In [6]:
print(torch.__version__)

1.13.0+cu117


In [7]:
torch.cuda.device_count()

1

In [8]:
!pip uninstall -y numpy
!pip install numpy==1.24.4

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
accelerate 1.5.2 requires torch>=2.0.0, but you have torch 1.13.0 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompati

In [9]:
import numpy as np

In [10]:
print(np.__version__)

1.24.4


In [11]:
!git clone https://github.com/mariamhash/stylegan2-ada-pytorch.git

fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.


In [12]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


In [24]:
import cv2
import numpy as np
import random
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.utils import save_image
import subprocess
import os

# Function to generate synthetic cracks using StyleGAN2-ADA
def generate_synthetic_cracks(model_path, output_dir, seeds="0-10"):
    os.makedirs(output_dir, exist_ok=True)
    command = [
        "python", "generate.py",
        "--outdir", output_dir,
        "--trunc", "1",
        "--seeds", seeds,
        "--network", model_path
    ]
    subprocess.run(command, check=True)
    return [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".png")]

# Load Binary Mask of Empty Spots
def load_binary_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    empty_spots = cv2.bitwise_not(binary_mask)
    return empty_spots

# Remove white borders from all sides of an image
def crop_white_borders(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(cv2.bitwise_not(thresh))
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        img = img[y:y+h, x:x+w]
    return img

# Load and Resize GAN-Generated Crack
def load_synthetic_crack(crack_path, size=(128, 128)):
    crack = cv2.imread(crack_path, cv2.IMREAD_UNCHANGED)
    crack = crop_white_borders(crack)
    crack = cv2.resize(crack, size)
    return crack

# Find Random Empty Spot to Place Crack
def find_empty_spot(empty_spots, crack_size):
    h, w = empty_spots.shape
    ch, cw = crack_size
    for _ in range(100):  # Try 100 random positions
        x = random.randint(0, w - cw)
        y = random.randint(0, h - ch)
        region = empty_spots[y:y+ch, x:x+cw]
        if np.mean(region) > 250:  # Check if mostly empty
            return x, y
    return None

# Poisson Blending for seamless integration
def poisson_blend(crack_patch, road_patch, mask):
    center = (road_patch.shape[1] // 2, road_patch.shape[0] // 2)
    blended = cv2.seamlessClone(crack_patch, road_patch, mask, center, cv2.MIXED_CLONE)
    return blended

# Main Function
def add_synthetic_cracks(road_image_path, mask_path, model_path, output_dir="generated_cracks"):
    road_image = cv2.imread(road_image_path)
    empty_spots = load_binary_mask(mask_path)
    crack_paths = generate_synthetic_cracks(model_path, output_dir)

    for crack_path in crack_paths:
        crack = load_synthetic_crack(crack_path)
        position = find_empty_spot(empty_spots, crack.shape[:2])
        if position:
            x, y = position
            road_patch = road_image[y:y+crack.shape[0], x:x+crack.shape[1]]
            crack_gray = cv2.cvtColor(crack, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(crack_gray, 10, 255, cv2.THRESH_BINARY)
            mask = cv2.merge([mask, mask, mask])
            blended_patch = poisson_blend(crack, road_patch, mask)
            road_image[y:y+crack.shape[0], x:x+crack.shape[1]] = blended_patch
    return road_image

# ------------------------------
# Example Usage
# ------------------------------
road_image_path = "/content/drive/MyDrive/Colab_Notebooks/ZAKA_AIC/Capstone_Project/Attachment/Copy_of_4.png"
mask_path = "/content/drive/MyDrive/Colab_Notebooks/ZAKA_AIC/Capstone_Project/Attachment/Mask_of_4.png"
model_path = "/content/drive/MyDrive/Colab_Notebooks/ZAKA_AIC/Capstone_Project/c2-network-snapshot-000120.pkl"

result_image = add_synthetic_cracks(road_image_path, mask_path, model_path)
cv2.imwrite("/content/drive/MyDrive/Colab_Notebooks/ZAKA_AIC/Capstone_Project/Attachment/result_with_synthetic_cracks_P1.png", result_image)


True